In [ ]:
from bayesian_optimization import BayesianOptimization
import pandas as pd 
from botorch.test_functions import Ackley
from botorch.test_functions import Levy 
from botorch.test_functions import Beale
from botorch.test_functions import DixonPrice
from botorch.test_functions import DropWave
from botorch.test_functions import Hartmann
from botorch.test_functions import StyblinskiTang
# Variables that determine the size of the dataset
N_INITIAL = 10
EPOCHS = 3
BATCH_SIZE = 10
DIM = 2
LOWER = -32.768
UPPER = 32.768

optimizer = BayesianOptimization(fun=Ackley(dim=DIM,negate=True), 
                                 batch_size=BATCH_SIZE, 
                                 dim=DIM, 
                                 epochs=EPOCHS, 
                                 n_init=N_INITIAL, 
                                 lower_bound=LOWER,
                                 upper_bound=UPPER,
                                 seed=5,
                                 acqf_type='qUCB')

x_max, y_max = optimizer.run()
data = optimizer.get_data()
full = optimizer.format(data, dim=DIM, n_init=N_INITIAL, batch_size=BATCH_SIZE, epochs=EPOCHS)
display(full)

In [ ]:
import torch 
import botorch as bo
import matplotlib.pyplot as plt
import numpy as np

X_cols = [
    'D(+)-Biotin',
    'D-Pantothenic acid calcium salt',
    'Folic acid',
    'myo-Inositol',
    'Thiaminhydrochloride',
    'Pyridoxinhydrochloride',
    'Niacin',
    '4-Aminobenzoic acid',
    'Riboflavin',
    'Cobalamin',
    'Boric acid',
    'Potassium iodide',
    'Iron(III) chloride',
    'Iron(II) sulfate heptahydrate',
    'Manganese(II) sulfate monohydrate',
    'Manganese(II) chloride tetrahydrate',
    'Sodium molybdat dihydrate',
    'Copper(II) sulfate pentahydrate',
    'Zinc sulfate heptahydrate',
    'Cobalt(II) chloride hexahydrate',
]
Y1_cols = ['Growth (OD)']
Y2_cols = ['Lipid content (FL)']

X = bo_data[X_cols]
y1 = bo_data[Y1_cols]
y2 = bo_data[Y2_cols]
X = torch.tensor(X.values, dtype=torch.double)
y1 = torch.tensor(y1.values, dtype=torch.double)
y2 = torch.tensor(y2.values, dtype=torch.double)

# Remove the first 20 rows
X = X[20:]
y1 = y1[20:]
y2 = y2[20:]

X_min, X_max = X.min(dim=0).values, X.max(dim=0).values # to [0,1]
X_normed = (X - X_min) / (X_max - X_min)

Y1_normed = (y1 - y1.mean()) / y1.std() # Norm to (0,1)
Y2_normed = (y2 - y2.mean()) / y2.std() # Norm to (0,1)

print(X.shape)
print(y1.shape)
gp1 = bo.models.SingleTaskGP(X_normed, Y1_normed) # Train the model Y1
gp2 = bo.models.SingleTaskGP(X_normed, Y2_normed) # Train the model Y2